In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In this notebook, we can estimate: how many visits could we have for every field, if we evenly split a typical total number of visits over a very large sky footprint? 

So first: estimate the total number of visits available. 

In [ ]:
# Total approximate number of visits in opsim:
totalNvis = 2400000
# Let's say we can play with 93% of these visits
totalNvis *= 0.93
print("The number of visits available is %d (%.2fM)" % (totalNvis, totalNvis/1000000))

Now we'll count the number of fields we want to include in this 'big sky' footprint, using the opsim tessellation. This is equivalent to estimating the number of pointings required to cover the footprint.

In [ ]:
# Read the field list.
fields = pd.read_csv('field_list.csv')
# what does the field list look like?
fields[0:5]

In [ ]:
# Select fields with declination between -90 and 32 -- our "big sky" footprint
bigsky = fields.query('(dec >= -90) and (dec <= 32)') 

In [ ]:
# Plot the fields so we can check what this footprint looks like.
def radec2project(ra, dec):
    return (np.radians(ra) - np.pi, np.radians(dec))

fig = plt.figure(figsize=(8, 8))
ax = plt.subplot(111, projection="aitoff")
x, y = radec2project(bigsky.ra, bigsky.dec)
ax.scatter(x, y, alpha=0.5)
plt.grid(True)

In [ ]:
# How many fields are included in this "big sky" version of the survey?
nfields = len(bigsky)
print("The number of fields in the footprint is %d" % (nfields))

So how many visits could we potentially have per field, if we split them evenly among fields?

In [ ]:
# Exact number of visits per field
visPerField = totalNvis / nfields
# Round the number of visits per field to an integer
visPerField = int(round(visPerField))
# And recalculate what this means for the totalNvis -- if the total # changed too much, maybe round down
propTotal = visPerField * nfields
print("This corresponds to %d visits per field" % (visPerField, ))
print("This implies a total number of visits required of %d (compared to original estimate of %d): %.2f%s change"
     % (propTotal, totalNvis, float(propTotal - totalNvis) / totalNvis * 100.0, "%"))

Let's add some estimate on the coadded depths you could achieve.

This requires assuming a dark sky, zenith, typical single visit depth and then scaling by the number of visits in each filter. 

In [ ]:
# Current expected performance
single_m5 = {'u': 23.98, 'g': 24.91, 'r': 24.42, 'i': 23.97, 'z': 23.38, 'y': 22.47}

In [ ]:
# How should we divide the visits? (per field) 
fractionsPerFilter = {'u': 0.12, 'g': 0.17, 'r': 0.23, 'i': 0.23, 'z': 0.25, 'z': 0.25}
fracSum = 0
for f, v in fractionsPerFilter.items():
    fracSum += v
    print('Fraction in %s: %.3f' % (f, v))
print("Total: %.3f" % (fracSum))

In [ ]:
# Make actual numbers per filter .. jiggle as necessary. (note that you could set these by hand instead)
visPerFilter = {}
fieldTotal = 0
for f in fractionsPerFilter:
    visPerFilter[f] = int(round(fractionsPerFilter[f] * visPerField))
    fieldTotal += visPerFilter[f]
    print('Visits in %s: %d' % (f, visPerFilter[f]))
print('Total per field: %d (compared to potential %d per field previously calculated)' %(fieldTotal, visPerField))

In [ ]:
# Jiggle by hand .. 
visPerFilter['g'] += 1
fieldTotal = 0
for f in visPerFilter:
    fieldTotal += visPerFilter[f]
    print('Visits in %s: %d' % (f, visPerFilter[f]))
print('Total per field: %d (compared to potential %d per field previously calculated)' %(fieldTotal, visPerField))

So calculate the scaled coadded depth per filter, per field. Note that this assumes all observations are taken under the same "dark-sky, zenith" conditions.

In [ ]:
coadd_m5 = {}
for f in visPerFilter:
    coadd_m5[f] = single_m5[f] + 2.5 * np.log10(np.sqrt(visPerFilter[f]))
    print("Coadded depth in %s: %.2f" % (f, coadd_m5[f]))

Given "realistic" opsim conditions, we would expect some typical offsets from these dark-sky, zenith coadded depths. 
Using the WFD region from kraken\_2026 we can calculate these expected offsets:

In [ ]:
# Using estimates from kraken_2026:
kraken_single_m5 = {'u': 23.78, 'g': 24.81, 'r': 24.35, 'i': 23.92, 'z': 23.34, 'y': 22.45}
visPerFilter = {'u': 64, 'g': 90, 'r': 206, 'i': 204, 'z': 186, 'y': 188}
opsimCoaddM5 = {'u': 25.65, 'g': 27.15, 'r': 27.20, 'i': 26.62, 'z': 25.72, 'y': 24.91}
kraken_coadd_m5 = {}
offset = {}
for f in visPerFilter:
    kraken_coadd_m5[f] = kraken_single_m5[f] + 2.5 * np.log10(np.sqrt(visPerFilter[f]))
    offset[f] = kraken_coadd_m5[f] - opsimCoaddM5[f]
    print("Coadded depth in %s: %.2f - correction is %.2f" % (f, kraken_coadd_m5[f], offset[f]))

In [ ]:
saved_offsets = {'u': 0.39, 'g': 0.10, 'r': 0.04, 'i': 0.19, 'z': 0.46, 'y': 0.38}